In [ ]:
# This notebook processes a Sentinel 2 L1C product in .SAFE format
# to a Sentinel 2 L2A products # subsetted to a rectangular window.
# A file named 'metadata.csv' must be placed in the export directory.
# This file is updated with metadata.
#
# This code is tested to work on Debian 10 Linux but should work in any Liux environment.
#
# The data in the directory of L2A products can be uploaded to Google Earth Engine.
# This should be done after a number of products are completed for efficiency.
#

In [ ]:
# User specified stuff

# This is the L1 file to be processed from the import dir, change this for each job you launch
L1Name = 'S2A_MSIL1C_20210805T180921_N0301_R084_T13UCS_20210805T215442.SAFE'

#version of sen2cor
s2cor = 'Sen2Cor-02.10.01-Linux64'
#sen2cor dir
sDirname = '/home/richard_fernandes/GEEwork/sen2cor/'

#import dir where the L1C SAFE product should be placed
iDirname = '/home/richard_fernandes/GEEwork/sen2cor/imports/'
# working dir where each job swill make its own work directory to put temp files, allows multiple jobs running at once
wDirname = '/home/richard_fernandes/GEEwork/sen2cor/working/'
# output dir where the L2A SAFE product will go
oDirname = '/home/richard_fernandes/GEEwork/sen2cor/output/'
#export dir where the geotiff file for upload to GEE will go
eDirname = '/home/richard_fernandes/GEEwork/sen2cor/exports/'


#L2a filename and subset region in degreess 
#enter rocWidth for entire granule
ccLat = 50.79; # degrees N
ccLong = -107.17; # degrees E
rocWidth = 0.5; # degrees
rocWin = [ccLong-rocWidth, ccLat+rocWidth, ccLong+rocWidth, ccLat-rocWidth] # the subset window in degrees




In [52]:
#imports
from osgeo import gdal
from osgeo import ogr
from osgeo import osr
from osgeo import gdal_array
from osgeo import gdalconst
from osgeo import gdalnumeric
from osgeo import gdalconst
import os
import glob
from shutil import rmtree
from pathlib import Path
import xml.etree.ElementTree as ET
from datetime import datetime
import csv

In [53]:
# Enable exceptions
gdal.UseExceptions()    

In [84]:
#make a work directory - this allows parallel processing by multiple script executions
workDirname  = wDirname+L1Name[0:len(L1Name)-5]+'/'
os.mkdir(workDirname) 

In [55]:
# run sen2cor and find the result file, should take <30minutes per granule
cmd = sDirname  + s2cor + "/bin/L2A_Process "+iDirname+L1Name+" --output_dir " + oDirname + " --work_dir " + workDirname
print(cmd)
os.system(cmd)


/home/richard_fernandes/GEEwork/sen2cor/Sen2Cor-02.10.01-Linux64/bin/L2A_Process /home/richard_fernandes/GEEwork/sen2cor/imports/S2A_MSIL1C_20210805T180921_N0301_R084_T13UCS_20210805T215442.SAFE --output_dir /home/richard_fernandes/GEEwork/sen2cor/output/ --work_dir /home/richard_fernandes/GEEwork/sen2cor/working/
Sen2Cor. Version: 02.10.01, created: 2021.12.13, supporting Level-1C product version 14.2 - 14.9 started ...
Product version: 14.6
Operation mode: TOOLBOX
Processing baseline: 99.99
Progress[%]:  0.00 : Generating datastrip metadata
L2A datastrip successfully generated
No resolution specified, will process 20 and 10 m resolution
20 m resolution will be downsampled to 60 m
Progress[%]: 0.06 : PID-24045, L2A_ProcessTile: processing with resolution 20 m, elapsed time[s]: 1.215, total: 0:00:05.679730
Progress[%]: 0.06 : PID-24045, L2A_ProcessTile: start of pre processing, elapsed time[s]: 0.001, total: 0:00:05.680368
Progress[%]: 0.07 : PID-24045, L2A_Tables: start import, elapse

0

In [57]:
# make the path for the output L2 product and for the output Geotiff version
L2path = oDirname+L1name[0:8]+'2A'+L1name[10:24]+'*.SAFE'
L2Name = glob.glob(oDirname+L1name[0:8]+'2A'+L1name[10:24]+'*.SAFE')[0]
exportName = (L2name[len(oDirname):(len(L2name)-5)])
print(L2Name)

/home/richard_fernandes/GEEwork/sen2cor/output/S2A_MSIL2A_20210805T180921_N9999_R084_T13UCS_20220128T020026.SAFE


In [58]:
# determine time between dateText and 1970,1,1,0,0,0 in milliseconds
def systemTime(dateText):
    date2 =  dateText;
    year = int(date2[0:4]);
    month = int(date2[5:7]);
    day = int(date2[8:10]);
    hour =  int(date2[11:13]);
    minute = int(date2[14:16]);
    second = int(date2[17:19]);
    systemTime= ((datetime(year,month,day,hour,minute,second)-datetime(1970,1,1,0,0,0))).total_seconds() * 1000.0
    return systemTime

In [59]:
def get_directory_size(directory):
    """Returns the `directory` size in bytes."""
    total = 0
    try:
        # print("[+] Getting the size of", directory)
        for entry in os.scandir(directory):
            if entry.is_file():
                # if it's a file, use stat() function
                total += entry.stat().st_size
            elif entry.is_dir():
                # if it's a directory, recursively call this function
                total += get_directory_size(entry.path)
    except NotADirectoryError:
        # if `directory` isn't a directory, get the file size then
        return os.path.getsize(directory)
    except PermissionError:
        # if for whatever reason we can't open the folder, return 0
        return 0
    return total

In [61]:
# Add a metadata row to metadata.csv
def addMetadata(L2Name,exportName,metadataName):
    #open the metadata files
    tree0= ET.parse(L2Name+'/MTD_MSIL2A.xml')
    root0= tree0.getroot()
    tree1= ET.parse(L2Name+'/'+root0[0][0][11][0][0][0].text[0:42]+'/MTD_TL.xml')
    root1= tree1.getroot()
    
    #parse the metadata files
    AOT_RETRIEVAL_ACCURACY= float(root0[3][3][14].text)
    CLOUDY_PIXEL_PERCENTAGE= float(root1[2][0][0].text)
    CLOUD_COVERAGE_ASSESSMENT= float(root0[3][0].text)
    CLOUDY_SHADOW_PERCENTAGE= float(root0[3][3][3].text)
    DARK_FEATURES_PERCENTAGE= float(root0[3][3][2].text)
    DATASTRIP_ID= root0[0][0][11][0][0].attrib
    DATASTRIP_ID= DATASTRIP_ID["datastripIdentifier"]
    DATATAKE_IDENTIFIER= root0[0][0][9].attrib
    DATATAKE_IDENTIFIER= DATATAKE_IDENTIFIER["datatakeIdentifier"]
    DATATAKE_TYPE= root0[0][0][9][1].text
    DEGRADED_MSI_DATA_PERCENTAGE= float(root1[2][0][1].text)
    FORMAT_CORRECTNESS= root0[3][2][0][0].text
    GENERAL_QUALITY= root0[3][2][0][1].text
    GENERATION_TIME= systemTime(root0[0][0][6].text)
    GEOMETRIC_QUALITY= root0[3][2][0][2].text
    GRANULE_ID= root0[0][0][11][0][0].attrib   
    GRANULE_ID= GRANULE_ID["granuleIdentifier"]
    HIGH_PROBA_CLOUDS_PERCENTAGE= float(root0[3][3][9].text)
    MEAN_INCIDENCE_AZIMUTH_ANGLE_B1= float(root1[1][1][93][0][1].text)
    MEAN_INCIDENCE_AZIMUTH_ANGLE_B2= float(root1[1][1][93][3][1].text)
    MEAN_INCIDENCE_AZIMUTH_ANGLE_B3= float(root1[1][1][93][4][1].text)
    MEAN_INCIDENCE_AZIMUTH_ANGLE_B4= float(root1[1][1][93][5][1].text)
    MEAN_INCIDENCE_AZIMUTH_ANGLE_B5= float(root1[1][1][93][6][1].text)
    MEAN_INCIDENCE_AZIMUTH_ANGLE_B6= float(root1[1][1][93][7][1].text)
    MEAN_INCIDENCE_AZIMUTH_ANGLE_B7= float(root1[1][1][93][8][1].text)
    MEAN_INCIDENCE_AZIMUTH_ANGLE_B8= float(root1[1][1][93][9][1].text)
    MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A= float(root1[1][1][93][10][1].text)
    MEAN_INCIDENCE_AZIMUTH_ANGLE_B9= float(root1[1][1][93][11][1].text)
    MEAN_INCIDENCE_AZIMUTH_ANGLE_B10= float(root1[1][1][93][12][1].text)
    MEAN_INCIDENCE_AZIMUTH_ANGLE_B11= float(root1[1][1][93][1][1].text)
    MEAN_INCIDENCE_AZIMUTH_ANGLE_B12= float(root1[1][1][93][2][1].text)
    MEAN_INCIDENCE_ZENITH_ANGLE_B1= float(root1[1][1][93][0][0].text)
    MEAN_INCIDENCE_ZENITH_ANGLE_B2= float(root1[1][1][93][3][0].text)
    MEAN_INCIDENCE_ZENITH_ANGLE_B3= float(root1[1][1][93][4][0].text)
    MEAN_INCIDENCE_ZENITH_ANGLE_B4= float(root1[1][1][93][5][0].text)
    MEAN_INCIDENCE_ZENITH_ANGLE_B5= float(root1[1][1][93][6][0].text)
    MEAN_INCIDENCE_ZENITH_ANGLE_B6= float(root1[1][1][93][7][0].text)
    MEAN_INCIDENCE_ZENITH_ANGLE_B7= float(root1[1][1][93][8][0].text)
    MEAN_INCIDENCE_ZENITH_ANGLE_B8= float(root1[1][1][93][9][0].text)
    MEAN_INCIDENCE_ZENITH_ANGLE_B8A= float(root1[1][1][93][10][0].text)
    MEAN_INCIDENCE_ZENITH_ANGLE_B9= float(root1[1][1][93][11][0].text)
    MEAN_INCIDENCE_ZENITH_ANGLE_B10= float(root1[1][1][93][12][0].text)
    MEAN_INCIDENCE_ZENITH_ANGLE_B11= float(root1[1][1][93][1][0].text)
    MEAN_INCIDENCE_ZENITH_ANGLE_B12= float(root1[1][1][93][1][0].text)
    MEAN_SOLAR_AZIMUTH_ANGLE= float(root1[1][1][1][1].text)
    MEAN_SOLAR_ZENITH_ANGLE= float(root1[1][1][1][0].text)
    MEDIUM_PROBA_CLOUDS_PERCENTAGE= float(root0[3][3][10].text)
    MGRS_TILE= root0[0][0][2].text
    MGRS_TILE= MGRS_TILE[39:44]
    NODATA_PIXEL_PERCENTAGE= float(root0[3][3][0].text)
    NOT_VEGETATED_PERCENTAGE= float(root0[3][3][5].text)
    PROCESSING_BASELINE= root0[0][0][5].text
    PRODUCT_ID= root0[0][0][2].text
    PRODUCT_ID= PRODUCT_ID[0:len(PRODUCT_ID)-5]
    RADIATIVE_TRANSFER_ACCURACY= float(root0[3][3][12].text)
    RADIOMETRIC_QUALITY= root0[3][2][0][3].text
    REFLECTANCE_CONVERSION_CORRECTION= root0[0][1][4][0].text
    SATURATED_DEFECTIVE_PIXEL_PERCENTAGE= float(root0[3][3][1].text)
    SENSING_ORBIT_DIRECTION= root0[0][0][9][4].text
    SENSING_ORBIT_NUMBER= float(root0[0][0][9][3].text)
    SENSOR_QUALITY= root0[3][2][0][4].text
    SOLAR_IRRADIANCE_B1= float(root0[0][1][4][1][0].text)
    SOLAR_IRRADIANCE_B2= float(root0[0][1][4][1][1].text)
    SOLAR_IRRADIANCE_B3= float(root0[0][1][4][1][2].text)
    SOLAR_IRRADIANCE_B4= float(root0[0][1][4][1][3].text)
    SOLAR_IRRADIANCE_B5= float(root0[0][1][4][1][4].text)
    SOLAR_IRRADIANCE_B6= float(root0[0][1][4][1][5].text)
    SOLAR_IRRADIANCE_B7= float(root0[0][1][4][1][6].text)
    SOLAR_IRRADIANCE_B8= float(root0[0][1][4][1][7].text)
    SOLAR_IRRADIANCE_B8A= float(root0[0][1][4][1][8].text)
    SOLAR_IRRADIANCE_B9= float(root0[0][1][4][1][9].text)
    SOLAR_IRRADIANCE_B10= float(root0[0][1][4][1][10].text)
    SOLAR_IRRADIANCE_B11= float(root0[0][1][4][1][11].text)
    SOLAR_IRRADIANCE_B12= float(root0[0][1][4][1][12].text)
    SNOW_ICE_PERCENTAGE= float(root0[3][3][11].text)
    SPACECRAFT_NAME= root0[0][0][9][0].text    
    THIN_CIRRUS_PERCENTAGE= float(root0[3][3][10].text)
    UNCLASSIFIED_PERCENTAGE= float(root0[3][3][7].text)
    VEGETATION_PERCENTAGE= float(root0[3][3][4].text)
    WATER_PERCENTAGE= float(root0[3][3][6].text)
    system_asset_size = get_directory_size(L2Name)
    system_footprint =  0
    system_index= root0[0][0][2].text
    system_index= system_index[0:37]
    system_time_end= systemTime(root0[0][0][0].text)
    system_time_start= systemTime(root0[0][0][1].text)
    
    # fieldnames
    fieldnames = [\
        'id_no', \
        'AOT_RETRIEVAL_ACCURACY', \
        'CLOUDY_PIXEL_PERCENTAGE', \
        'CLOUD_COVERAGE_ASSESSMENT', \
        'CLOUDY_SHADOW_PERCENTAGE', \
        'DARK_FEATURES_PERCENTAGE', \
        'DATASTRIP_ID', \
        'DATATAKE_IDENTIFIER', \
        'DATATAKE_TYPE', \
        'DEGRADED_MSI_DATA_PERCENTAGE', \
        'FORMAT_CORRECTNESS', \
        'GENERAL_QUALITY', \
        'GENERATION_TIME', \
        'GEOMETRIC_QUALITY', \
        'GRANULE_ID', \
        'HIGH_PROBA_CLOUDS_PERCENTAGE', \
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B1', \
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B2', \
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B3', \
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B4', \
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B5',\
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B6',\
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B7',\
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8',\
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A',\
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B9',\
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B10',\
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B11',\
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B12',\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B1',\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B2',\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B3',\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B4',\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B5',\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B6',\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B7',\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B8',\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B8A',\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B9',\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B10',\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B11',\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B12',\
        'MEAN_SOLAR_AZIMUTH_ANGLE',\
        'MEAN_SOLAR_ZENITH_ANGLE',\
        'MEDIUM_PROBA_CLOUDS_PERCENTAGE',\
        'MGRS_TILE',\
        'NODATA_PIXEL_PERCENTAGE',\
        'NOT_VEGETATED_PERCENTAGE',\
        'PROCESSING_BASELINE',\
        'PRODUCT_ID',\
        'RADIATIVE_TRANSFER_ACCURACY',\
        'RADIOMETRIC_QUALITY',\
        'REFLECTANCE_CONVERSION_CORRECTION',\
        'SATURATED_DEFECTIVE_PIXEL_PERCENTAGE',\
        'SENSING_ORBIT_DIRECTION',\
        'SENSING_ORBIT_NUMBER',\
        'SENSOR_QUALITY',\
        'SOLAR_IRRADIANCE_B1',\
        'SOLAR_IRRADIANCE_B2',\
        'SOLAR_IRRADIANCE_B3',\
        'SOLAR_IRRADIANCE_B4',\
        'SOLAR_IRRADIANCE_B5',\
        'SOLAR_IRRADIANCE_B6',\
        'SOLAR_IRRADIANCE_B7',\
        'SOLAR_IRRADIANCE_B8',\
        'SOLAR_IRRADIANCE_B8A',\
        'SOLAR_IRRADIANCE_B9',\
        'SOLAR_IRRADIANCE_B10',\
        'SOLAR_IRRADIANCE_B11',\
        'SOLAR_IRRADIANCE_B12',\
        'SNOW_ICE_PERCENTAGE',\
        'SPACECRAFT_NAME',\
        'THIN_CIRRUS_PERCENTAGE',\
        'UNCLASSIFIED_PERCENTAGE',\
        'VEGETATION_PERCENTAGE',\
        'WATER_PERCENTAGE',\
        'system_asset_size',\
        'system_footprint',\
        'system_index',\
        'system_time_end',\
        'system_time_start']     
    
    #write a row of metadata
    with open(metadataName, 'a', newline='') as csvfile:  
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow({\
        'id_no':exportName,\
        'AOT_RETRIEVAL_ACCURACY':AOT_RETRIEVAL_ACCURACY, \
        'CLOUDY_PIXEL_PERCENTAGE':CLOUDY_PIXEL_PERCENTAGE, \
        'CLOUD_COVERAGE_ASSESSMENT':CLOUD_COVERAGE_ASSESSMENT, \
        'CLOUDY_SHADOW_PERCENTAGE':CLOUDY_SHADOW_PERCENTAGE, \
        'DARK_FEATURES_PERCENTAGE':DARK_FEATURES_PERCENTAGE, \
        'DATASTRIP_ID':DATASTRIP_ID, \
        'DATATAKE_IDENTIFIER':DATATAKE_IDENTIFIER, \
        'DATATAKE_TYPE':DATATAKE_TYPE, \
        'DEGRADED_MSI_DATA_PERCENTAGE':DEGRADED_MSI_DATA_PERCENTAGE, \
        'FORMAT_CORRECTNESS':FORMAT_CORRECTNESS, \
        'GENERAL_QUALITY':GENERAL_QUALITY, \
        'GENERATION_TIME':GENERATION_TIME, \
        'GEOMETRIC_QUALITY':GEOMETRIC_QUALITY, \
        'GRANULE_ID':GRANULE_ID, \
        'HIGH_PROBA_CLOUDS_PERCENTAGE':HIGH_PROBA_CLOUDS_PERCENTAGE, \
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B1':MEAN_INCIDENCE_AZIMUTH_ANGLE_B1, \
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B2':MEAN_INCIDENCE_AZIMUTH_ANGLE_B2, \
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B3':MEAN_INCIDENCE_AZIMUTH_ANGLE_B3, \
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B4':MEAN_INCIDENCE_AZIMUTH_ANGLE_B4, \
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B5':MEAN_INCIDENCE_AZIMUTH_ANGLE_B5,\
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B6':MEAN_INCIDENCE_AZIMUTH_ANGLE_B6,\
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B7':MEAN_INCIDENCE_AZIMUTH_ANGLE_B7,\
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8':MEAN_INCIDENCE_AZIMUTH_ANGLE_B8,\
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A':MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A,\
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B9':MEAN_INCIDENCE_AZIMUTH_ANGLE_B9,\
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B10':MEAN_INCIDENCE_AZIMUTH_ANGLE_B10,\
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B11':MEAN_INCIDENCE_AZIMUTH_ANGLE_B11,\
        'MEAN_INCIDENCE_AZIMUTH_ANGLE_B12':MEAN_INCIDENCE_AZIMUTH_ANGLE_B12,\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B1':MEAN_INCIDENCE_ZENITH_ANGLE_B1,\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B2':MEAN_INCIDENCE_ZENITH_ANGLE_B2,\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B3':MEAN_INCIDENCE_ZENITH_ANGLE_B3,\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B4':MEAN_INCIDENCE_ZENITH_ANGLE_B4,\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B5':MEAN_INCIDENCE_ZENITH_ANGLE_B5,\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B6':MEAN_INCIDENCE_ZENITH_ANGLE_B6,\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B7':MEAN_INCIDENCE_ZENITH_ANGLE_B7,\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B8':MEAN_INCIDENCE_ZENITH_ANGLE_B8,\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B8A':MEAN_INCIDENCE_ZENITH_ANGLE_B8A,\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B9':MEAN_INCIDENCE_ZENITH_ANGLE_B9,\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B10':MEAN_INCIDENCE_ZENITH_ANGLE_B10,\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B11':MEAN_INCIDENCE_ZENITH_ANGLE_B11,\
        'MEAN_INCIDENCE_ZENITH_ANGLE_B12':MEAN_INCIDENCE_ZENITH_ANGLE_B12,\
        'MEAN_SOLAR_AZIMUTH_ANGLE':MEAN_SOLAR_AZIMUTH_ANGLE,\
        'MEAN_SOLAR_ZENITH_ANGLE':MEAN_SOLAR_ZENITH_ANGLE,\
        'MEDIUM_PROBA_CLOUDS_PERCENTAGE':MEDIUM_PROBA_CLOUDS_PERCENTAGE,\
        'MGRS_TILE':MGRS_TILE,\
        'NODATA_PIXEL_PERCENTAGE':NODATA_PIXEL_PERCENTAGE,\
        'NOT_VEGETATED_PERCENTAGE':NOT_VEGETATED_PERCENTAGE,\
        'PROCESSING_BASELINE':PROCESSING_BASELINE,\
        'PRODUCT_ID':PRODUCT_ID,\
        'RADIATIVE_TRANSFER_ACCURACY':RADIATIVE_TRANSFER_ACCURACY,\
        'RADIOMETRIC_QUALITY':RADIOMETRIC_QUALITY,\
        'REFLECTANCE_CONVERSION_CORRECTION':REFLECTANCE_CONVERSION_CORRECTION,\
        'SATURATED_DEFECTIVE_PIXEL_PERCENTAGE':SATURATED_DEFECTIVE_PIXEL_PERCENTAGE,\
        'SENSING_ORBIT_DIRECTION':SENSING_ORBIT_DIRECTION,\
        'SENSING_ORBIT_NUMBER':SENSING_ORBIT_NUMBER,\
        'SENSOR_QUALITY':SENSOR_QUALITY,\
        'SOLAR_IRRADIANCE_B1':SOLAR_IRRADIANCE_B1,\
        'SOLAR_IRRADIANCE_B2':SOLAR_IRRADIANCE_B2,\
        'SOLAR_IRRADIANCE_B3':SOLAR_IRRADIANCE_B3,\
        'SOLAR_IRRADIANCE_B4':SOLAR_IRRADIANCE_B4,\
        'SOLAR_IRRADIANCE_B5':SOLAR_IRRADIANCE_B5,\
        'SOLAR_IRRADIANCE_B6':SOLAR_IRRADIANCE_B6,\
        'SOLAR_IRRADIANCE_B7':SOLAR_IRRADIANCE_B7,\
        'SOLAR_IRRADIANCE_B8':SOLAR_IRRADIANCE_B8,\
        'SOLAR_IRRADIANCE_B8A':SOLAR_IRRADIANCE_B8A,\
        'SOLAR_IRRADIANCE_B9':SOLAR_IRRADIANCE_B9,\
        'SOLAR_IRRADIANCE_B10':SOLAR_IRRADIANCE_B10,\
        'SOLAR_IRRADIANCE_B11':SOLAR_IRRADIANCE_B11,\
        'SOLAR_IRRADIANCE_B12':SOLAR_IRRADIANCE_B12,\
        'SNOW_ICE_PERCENTAGE':SNOW_ICE_PERCENTAGE,\
        'SPACECRAFT_NAME':SPACECRAFT_NAME,\
        'THIN_CIRRUS_PERCENTAGE':THIN_CIRRUS_PERCENTAGE,\
        'UNCLASSIFIED_PERCENTAGE':UNCLASSIFIED_PERCENTAGE,\
        'VEGETATION_PERCENTAGE':VEGETATION_PERCENTAGE,\
        'WATER_PERCENTAGE':WATER_PERCENTAGE,\
        'system_asset_size':system_asset_size,\
        'system_footprint':system_footprint,\
        'system_index':system_index,\
        'system_time_end':system_time_end,\
        'system_time_start':system_time_start
                            })

In [62]:
#add a row to the metadata.csv file

addMetadata(L2Name,exportName,eDirname+'metadata.csv')


In [68]:
gdal.Info(L2Name+'/MTD_MSIL2A.xml')

'Driver: SENTINEL2/Sentinel 2\nFiles: /home/richard_fernandes/GEEwork/sen2cor/output/S2A_MSIL2A_20210805T180921_N9999_R084_T13UCS_20220128T020026.SAFE/MTD_MSIL2A.xml\nSize is 512, 512\nMetadata:\n  AOT_QUANTIFICATION_VALUE=1000.0\n  AOT_QUANTIFICATION_VALUE_UNIT=none\n  AOT_RETRIEVAL_ACCURACY=0.0\n  BOA_QUANTIFICATION_VALUE=10000\n  BOA_QUANTIFICATION_VALUE_UNIT=none\n  CLOUD_COVERAGE_ASSESSMENT=2.748362\n  CLOUD_SHADOW_PERCENTAGE=0.436850\n  DARK_FEATURES_PERCENTAGE=0.691103\n  DATATAKE_1_DATATAKE_SENSING_START=2021-08-05T18:09:21.024Z\n  DATATAKE_1_DATATAKE_TYPE=INS-NOBS\n  DATATAKE_1_ID=GS2A_20210805T180921_031970_N99.99\n  DATATAKE_1_SENSING_ORBIT_DIRECTION=ASCENDING\n  DATATAKE_1_SENSING_ORBIT_NUMBER=84\n  DATATAKE_1_SPACECRAFT_NAME=Sentinel-2A\n  DEGRADED_ANC_DATA_PERCENTAGE=0.0\n  DEGRADED_MSI_DATA_PERCENTAGE=0\n  FOOTPRINT=POLYGON((-107.8763 51.415883589517534, -106.29791 51.44399790803582, -106.270706 50.45686117062749, -107.81607 50.42971299170202, -107.8763 51.41588358951753

In [80]:
# Open L2a 10m dataset and print info
f10m = gdal.Open('SENTINEL2_L2A:'+L2Name+'/MTD_MSIL2A.xml'+':10m:EPSG_32613')
gdal.Info(f10m)

'Driver: SENTINEL2/Sentinel 2\nFiles: /home/richard_fernandes/GEEwork/sen2cor/output/S2A_MSIL2A_20210805T180921_N9999_R084_T13UCS_20220128T020026.SAFE/MTD_MSIL2A.xml\n       /home/richard_fernandes/GEEwork/sen2cor/output/S2A_MSIL2A_20210805T180921_N9999_R084_T13UCS_20220128T020026.SAFE/GRANULE/L2A_T13UCS_A031970_20210805T180919/MTD_TL.xml\n       /home/richard_fernandes/GEEwork/sen2cor/output/S2A_MSIL2A_20210805T180921_N9999_R084_T13UCS_20220128T020026.SAFE/GRANULE/L2A_T13UCS_A031970_20210805T180919/IMG_DATA/R10m/T13UCS_20210805T180921_B04_10m.jp2\n       /home/richard_fernandes/GEEwork/sen2cor/output/S2A_MSIL2A_20210805T180921_N9999_R084_T13UCS_20220128T020026.SAFE/GRANULE/L2A_T13UCS_A031970_20210805T180919/IMG_DATA/R10m/T13UCS_20210805T180921_B03_10m.jp2\n       /home/richard_fernandes/GEEwork/sen2cor/output/S2A_MSIL2A_20210805T180921_N9999_R084_T13UCS_20220128T020026.SAFE/GRANULE/L2A_T13UCS_A031970_20210805T180919/IMG_DATA/R10m/T13UCS_20210805T180921_B02_10m.jp2\n       /home/richar

In [86]:
# Open L2a 20m dataset and print info
f20m = gdal.Open('SENTINEL2_L2A:'+L2Name+'/MTD_MSIL2A.xml'+':20m:EPSG_32613')
gdal.Info(f20m)

'Driver: SENTINEL2/Sentinel 2\nFiles: /home/richard_fernandes/GEEwork/sen2cor/output/S2A_MSIL2A_20210805T180921_N9999_R084_T13UCS_20220128T020026.SAFE/MTD_MSIL2A.xml\n       /home/richard_fernandes/GEEwork/sen2cor/output/S2A_MSIL2A_20210805T180921_N9999_R084_T13UCS_20220128T020026.SAFE/GRANULE/L2A_T13UCS_A031970_20210805T180919/MTD_TL.xml\n       /home/richard_fernandes/GEEwork/sen2cor/output/S2A_MSIL2A_20210805T180921_N9999_R084_T13UCS_20220128T020026.SAFE/GRANULE/L2A_T13UCS_A031970_20210805T180919/IMG_DATA/R20m/T13UCS_20210805T180921_B05_20m.jp2\n       /home/richard_fernandes/GEEwork/sen2cor/output/S2A_MSIL2A_20210805T180921_N9999_R084_T13UCS_20220128T020026.SAFE/GRANULE/L2A_T13UCS_A031970_20210805T180919/IMG_DATA/R20m/T13UCS_20210805T180921_B06_20m.jp2\n       /home/richard_fernandes/GEEwork/sen2cor/output/S2A_MSIL2A_20210805T180921_N9999_R084_T13UCS_20220128T020026.SAFE/GRANULE/L2A_T13UCS_A031970_20210805T180919/IMG_DATA/R20m/T13UCS_20210805T180921_B07_20m.jp2\n       /home/richar

In [87]:
# Open L2a 60m dataset and print info
f60m = gdal.Open('SENTINEL2_L2A:'+L2Name+'/MTD_MSIL2A.xml'+':60m:EPSG_32613')
gdal.Info(f60m)

'Driver: SENTINEL2/Sentinel 2\nFiles: /home/richard_fernandes/GEEwork/sen2cor/output/S2A_MSIL2A_20210805T180921_N9999_R084_T13UCS_20220128T020026.SAFE/MTD_MSIL2A.xml\n       /home/richard_fernandes/GEEwork/sen2cor/output/S2A_MSIL2A_20210805T180921_N9999_R084_T13UCS_20220128T020026.SAFE/GRANULE/L2A_T13UCS_A031970_20210805T180919/MTD_TL.xml\n       /home/richard_fernandes/GEEwork/sen2cor/output/S2A_MSIL2A_20210805T180921_N9999_R084_T13UCS_20220128T020026.SAFE/GRANULE/L2A_T13UCS_A031970_20210805T180919/IMG_DATA/R60m/T13UCS_20210805T180921_B01_60m.jp2\n       /home/richard_fernandes/GEEwork/sen2cor/output/S2A_MSIL2A_20210805T180921_N9999_R084_T13UCS_20220128T020026.SAFE/GRANULE/L2A_T13UCS_A031970_20210805T180919/IMG_DATA/R60m/T13UCS_20210805T180921_B09_60m.jp2\n       /home/richard_fernandes/GEEwork/sen2cor/output/S2A_MSIL2A_20210805T180921_N9999_R084_T13UCS_20220128T020026.SAFE/GRANULE/L2A_T13UCS_A031970_20210805T180919/IMG_DATA/R60m/T13UCS_20210805T180921_AOT_60m.jp2\n       /home/richar

In [88]:
# Export 10m version of L2A product (no ancillary bands) and output file describing contents
gdal.Translate(workDirname+'b4.tif',f10m,bandList=[1],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(workDirname+'b3.tif',f10m,bandList=[2],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(workDirname+'b2.tif',f10m,bandList=[3],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(workDirname+'b8.tif',f10m,bandList=[4],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(workDirname+'b5.tif',f20m,bandList=[1],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(workDirname+'b6.tif',f20m,bandList=[2],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(workDirname+'b7.tif',f20m,bandList=[3],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(workDirname+'b8a.tif',f20m,bandList=[4],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(workDirname+'b11.tif',f20m,bandList=[5],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(workDirname+'b12.tif',f20m,bandList=[6],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(workDirname+'b1.tif',f60m,bandList=[1],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(workDirname+'b9.tif',f60m,bandList=[1],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)

vrtOptions = gdal.BuildVRTOptions(separate=True)
gdal.BuildVRT(workDirname+'merged.vrt',[workDirname+'b1.tif',workDirname+'b2.tif',workDirname+'b3.tif',workDirname+'b4.tif', \
                                     workDirname+'b5.tif',workDirname+'b6.tif',workDirname+'b7.tif',workDirname+'b8.tif', \
                                     workDirname+'b8a.tif',workDirname+'b9.tif',workDirname+'b11.tif',workDirname+'b12.tif'] \
                                   ,options=vrtOptions)
gdal.Translate(eDirname+exportName+'.tif',workDirname+'merged.vrt')
gdal.Info(workDirname+'merged.vrt')


'Driver: VRT/Virtual Raster\nFiles: /home/richard_fernandes/GEEwork/sen2cor/working/S2A_MSIL1C_20210805T180921_N0301_R084_T13UCS_20210805T215442/merged.vrt\n       /home/richard_fernandes/GEEwork/sen2cor/working/S2A_MSIL1C_20210805T180921_N0301_R084_T13UCS_20210805T215442/b1.tif\n       /home/richard_fernandes/GEEwork/sen2cor/working/S2A_MSIL1C_20210805T180921_N0301_R084_T13UCS_20210805T215442/b2.tif\n       /home/richard_fernandes/GEEwork/sen2cor/working/S2A_MSIL1C_20210805T180921_N0301_R084_T13UCS_20210805T215442/b3.tif\n       /home/richard_fernandes/GEEwork/sen2cor/working/S2A_MSIL1C_20210805T180921_N0301_R084_T13UCS_20210805T215442/b4.tif\n       /home/richard_fernandes/GEEwork/sen2cor/working/S2A_MSIL1C_20210805T180921_N0301_R084_T13UCS_20210805T215442/b5.tif\n       /home/richard_fernandes/GEEwork/sen2cor/working/S2A_MSIL1C_20210805T180921_N0301_R084_T13UCS_20210805T215442/b6.tif\n       /home/richard_fernandes/GEEwork/sen2cor/working/S2A_MSIL1C_20210805T180921_N0301_R084_T13UCS

In [ ]:

#delete working directory
rmtree(workDirname)